# Chunking 전략 비교

**Chunking Strategy: Chonkie Library 를 기반**
   - https://docs.chonkie.ai/oss/chunkers/overview, https://docs.chonkie.ai/oss/chunkers/slumber-chunker, https://github.com/chonkie-inc/chonkie  
   - SlumberChunker (LLM 기반 Contextual Chunking - 의미를 채워서 Context 구성) 등

[Docling 에서도 HybridChunker 를 지원합니다](https://docling-project.github.io/docling/examples/hybrid_chunking/)

## 사전 준비

1. `pip install 'chunkie[all]'`
2. 실습용 PDF 파일

In [2]:
# ----------------------------------------------------------------------------
# OpenAI / OpenRouter 모델 초기화 헬퍼
# ----------------------------------------------------------------------------
import os
from typing import Literal

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()


def _resolve_api_context() -> tuple[str, str]:
    """선택된 API 키와 베이스 URL 정보를 반환합니다."""
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY가 필요합니다.")

    base_url = os.getenv("OPENROUTER_API_BASE") or "https://openrouter.ai/api/v1"

    return (api_key, base_url)


def create_openrouter_llm(
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.3,
    max_tokens: int | None = None,
    **kwargs: object,
) -> ChatOpenAI:
    """OpenAI 호환 LLM 생성 헬퍼.

    Args:
        model: 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/gpt-4o, anthropic/claude-3-sonnet, google/gemini-pro)
        temperature: 생성 온도 (0.0-2.0)
        max_tokens: 최대 생성 토큰 수

    Returns:
        ChatOpenAI: 설정된 LLM 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    openai_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "temperature": temperature,
        "max_retries": 3,
        "timeout": 60,
        **kwargs,
    }
    if max_tokens is not None:
        openai_kwargs["max_tokens"] = max_tokens
    if base_url:
        openai_kwargs["base_url"] = base_url
    return ChatOpenAI(**openai_kwargs)


def create_embedding_model(
    model: str = "openai/text-embedding-3-small",
    **kwargs,
) -> OpenAIEmbeddings:
    """OpenAI 호환 임베딩 모델 생성.

    Args:
        model: 임베딩 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/text-embedding-3-small, openai/text-embedding-3-large)
        **kwargs: 추가 파라미터 (encoding_format 등은 model_kwargs로 전달됨)

    Returns:
        OpenAIEmbeddings: 설정된 임베딩 모델 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    # 전달받은 kwargs에서 model_kwargs로 전달할 파라미터 분리
    # encoding_format, extra_headers 등은 model_kwargs로 전달
    model_kwargs: dict = {}
    embedding_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "show_progress_bar": True,
        "skip_empty": True,
    }

    # 전달받은 kwargs 처리
    for key, value in kwargs.items():
        # OpenRouter API 특정 파라미터는 model_kwargs로 전달
        if key in ("encoding_format"):
            model_kwargs[key] = value
        else:
            # 나머지는 OpenAIEmbeddings에 직접 전달
            embedding_kwargs[key] = value

    if base_url:
        embedding_kwargs["base_url"] = base_url

    # model_kwargs가 있으면 전달
    if model_kwargs:
        embedding_kwargs["model_kwargs"] = model_kwargs

    return OpenAIEmbeddings(**embedding_kwargs)


def create_embedding_model_direct(
    model: str = "qwen/qwen3-embedding-0.6b",
    encoding_format: Literal["float", "base64"] = "float",
    input_text: str | list[str] = "",
    **kwargs,
) -> list[float] | list[list[float]]:
    """OpenAI SDK를 직접 사용하여 임베딩 생성 (encoding_format 지원).

    LangChain의 OpenAIEmbeddings가 encoding_format을 지원하지 않을 때 사용.

    Args:
        model: 임베딩 모델 이름
        encoding_format: 인코딩 형식 ("float")
        input_text: 임베딩할 텍스트 (문자열 또는 문자열 리스트)
        **kwargs: 추가 파라미터

    Returns:
        임베딩 벡터 리스트 (단일 텍스트) 또는 리스트의 리스트 (여러 텍스트)
    """
    from openai import OpenAI

    api_key, base_url = _resolve_api_context()

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

    # input_text가 비어있으면 kwargs에서 가져오기
    if not input_text:
        input_text = kwargs.get("input", "")

    response = client.embeddings.create(
        model=model,
        input=input_text,
        encoding_format=encoding_format,
    )

    # 단일 텍스트인 경우 첫 번째 임베딩 반환
    if isinstance(input_text, str):
        return response.data[0].embedding
    else:
        # 여러 텍스트인 경우 모든 임베딩 반환
        return [item.embedding for item in response.data]


def get_available_model_types() -> dict[str, list[str]]:
    """OpenRouter에서 사용 가능한 모델 유형을 반환합니다.

    Returns:
        dict[str, list[str]]: 모델 유형별 모델 목록
    """
    return {
        "chat": [
            "openai/gpt-4.1",
            "openai/gpt-4.1-mini",
            "openai/gpt-5",
            "openai/gpt-5-mini",
            "anthropic/claude-sonnet-4.5",
            "anthropic/claude-haiku-4.5",
            "google/gemini-2.5-flash-preview-09-2025",
            "google/gemini-pro-2.5",
            "x-ai/grok-4-fast",
            "moonshotai/kimi-k2-thinking",
            "liquid/lfm-2.2-6b",
            "z-ai/glm-4.6",
        ],
        "embedding": [
            "openai/text-embedding-3-small",
            "openai/text-embedding-3-large",
            "google/gemini-embedding-001",
            "qwen/qwen3-embedding-0.6b",
            "qwen/qwen3-embedding-4b",
            "qwen/qwen3-embedding-8b",
        ],
    }


embeddings = create_embedding_model()
llm = create_openrouter_llm()

In [6]:
# ============================================================================
# Chunking 전략 비교
# ============================================================================

# NOTE: Parsing 완료된 문서를 Load 해와야함
from pathlib import Path

chunks_dir = Path("./chunks")
suffix = "_docling"

# NOTE: 파일 이름에 따라 파일 읽기
for md_path in chunks_dir.glob(f"*{suffix}.md"):
    with open(md_path) as f:
        markdown_text = f.read()

print(f"읽어온 파일 길이 확인: {len(markdown_text)}")

# ----------------------------------------------------------------------------
# 1. Chonkie SemanticChunker
# ----------------------------------------------------------------------------
print("\n[2/4] SemanticChunker (의미 유사도 기반)")
try:
    from chonkie import SemanticChunker

    embeddings = create_embedding_model(model="openai/text-embedding-3-small")
    chunker = SemanticChunker(
        embedding_model=embeddings,
        chunk_size=512,
        threshold=0.5,
    )

    chunks = chunker.chunk(markdown_text)

    print(f" 생성된 청크 수: {len(chunks)}")
    print("   첫 청크 미리보기 (처음 200자):")
    print("-" * 80)
    print(chunks[0].text[:200] if hasattr(chunks[0], "text") else str(chunks[0])[:200])
    print("...")
    print("-" * 80)

except ImportError:
    print("Chonkie가 설치되지 않았습니다.")
except Exception as e:
    print(f"오류: {e}")

# ----------------------------------------------------------------------------
# 3. LangChain RecursiveCharacterTextSplitter
# ----------------------------------------------------------------------------
print("\n[3/4] LangChain RecursiveCharacterTextSplitter (계층적)")
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=128,
        separators=["\n\n", "\n", " ", ""],  # 우선순위대로 분할
    )

    chunks = splitter.split_text(markdown_text)

    print(f" 생성된 청크 수: {len(chunks)}")
    print("   첫 청크 미리보기 (처음 200자):")
    print("-" * 80)
    print(chunks[0][:200])
    print("...")
    print("-" * 80)

except Exception as e:
    print(f"오류: {e}")

# ----------------------------------------------------------------------------
# 4. LangChain MarkdownTextSplitter
# ----------------------------------------------------------------------------
print("\n[4/4] LangChain MarkdownTextSplitter (마크다운 구조 인식)")
try:
    from langchain_text_splitters import MarkdownTextSplitter

    splitter = MarkdownTextSplitter(
        chunk_size=512,
        chunk_overlap=0,
    )

    chunks = splitter.split_text(markdown_text)

    print(f" 생성된 청크 수: {len(chunks)}")
    print("   첫 청크 미리보기 (처음 200자):")
    print("-" * 80)
    print(chunks[0][:200])
    print("...")
    print("-" * 80)

except Exception as e:
    print(f"오류: {e}")

읽어온 파일 길이 확인: 38723

[2/4] SemanticChunker (의미 유사도 기반)
오류: embedding_model must be a string or a BaseEmbeddings object

[3/4] LangChain RecursiveCharacterTextSplitter (계층적)
 생성된 청크 수: 107
   첫 청크 미리보기 (처음 200자):
--------------------------------------------------------------------------------
근로기준법

근로기준법

[시행 2025. 2. 23.] [법률 제20520호, 2024. 10. 22., 일부개정]

고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 
...
--------------------------------------------------------------------------------

[4/4] LangChain MarkdownTextSplitter (마크다운 구조 인식)
 생성된 청크 수: 97
   첫 청크 미리보기 (처음 200자):
--------------------------------------------------------------------------------
근로기준법

근로기준법

[시행 2025. 2. 23.] [법률 제20520호, 2024. 10. 22., 일부개정]

고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 
...
----------------------------------------------

---

## Chonkie 라이브러리 소개

Chonkie는 최신 RAG 파이프라인을 위한 포괄적인 청킹 라이브러리로, 다양한 구성요소를 제공합니다.

### Chonkie 구성요소

**1. Chefs (입력 정규화)**
- 원시 입력 데이터를 정규화하고 전처리
- TableChef, TextChef, MarkdownChef 등

**2. Fetchers (파일/텍스트 수집)**
- 다양한 소스에서 데이터 수집
- FileFetcher 등

**3. Chunkers (분할)**
- 다양한 전략으로 텍스트 분할
- TokenChunker, SentenceChunker, RecursiveChunker, SemanticChunker
- LateChunker, NeuralChunker, SlumberChunker, TableChunker
- **Experimental**: CodeChunker (코드 전용)

**4. Embeddings (임베딩)**
- 다양한 임베딩 모델 지원
- OpenAIEmbeddings, CohereEmbeddings, GeminiEmbeddings, JinaEmbeddings, VoyageAI
- Model2Vec, SentenceTransformer
- **Custom: TEI(Hugging Face Text Embeddings Inference) 연동 가능**

**5. Refinery (문맥 보강)**
- 청크에 추가 문맥 정보 부착
- OverlapRefinery: 오버랩으로 문맥 보강
- EmbeddingsRefinery: 임베딩 벡터 부착

**6. Handshakes (벡터DB 연동)**
- 다양한 벡터 데이터베이스 통합
- QdrantHandshake, ChromaHandshake, WeaviateHandshake
- TurbopufferHandshake, PineconeHandshake, PgvectorHandshake
- MongoDBHandshake, ElasticsearchHandshake

**7. Porters (입출력)**
- 청크 데이터 저장/로드
- JSONPorter, DatasetsPorter

**8. Utils (유틸리티)**
- Visualizer: 청크 시각화
- Hubbie: 헬퍼 함수

### 참고 문서

- Chunkers 개요: [https://docs.chonkie.ai/oss/chunkers/overview](https://docs.chonkie.ai/oss/chunkers/overview)
- OpenAI 임베딩: [https://docs.chonkie.ai/oss/embeddings/openai-embeddings](https://docs.chonkie.ai/oss/embeddings/openai-embeddings)
- 커스텀 임베딩(TEI): [https://docs.chonkie.ai/oss/embeddings/custom-embeddings](https://docs.chonkie.ai/oss/embeddings/custom-embeddings)
- Refinery 개요: [https://docs.chonkie.ai/oss/refinery/overview](https://docs.chonkie.ai/oss/refinery/overview)
- Qdrant Handshake: [https://docs.chonkie.ai/oss/handshakes/qdrant-handshake](https://docs.chonkie.ai/oss/handshakes/qdrant-handshake)
- JSONPorter: [https://docs.chonkie.ai/oss/porters/json-porter](https://docs.chonkie.ai/oss/porters/json-porter)
- Visualizer: [https://docs.chonkie.ai/oss/utils/visualizer](https://docs.chonkie.ai/oss/utils/visualizer)
- Hubbie: [https://docs.chonkie.ai/oss/utils/hubbie](https://docs.chonkie.ai/oss/utils/hubbie)
- Experimental Code Chunker: [https://docs.chonkie.ai/oss/experimental/code-chunker](https://docs.chonkie.ai/oss/experimental/code-chunker)


In [25]:
# ============================================================================
# Chunkie - OpenAI Embeddings
# ============================================================================
import tiktoken
from chonkie.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

# Chunkie - OpenAIEmbeddings 초기화
chunkie_openai_embeddins = OpenAIEmbeddings(
    base_url=os.getenv("OPENROUTER_BASE_URL"),
    model="openai/text-embedding-3-small",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    tokenizer=tiktoken.encoding_for_model("text-embedding-3-small"),
    dimension=1536,
    max_tokens=8192,
)
print(" OpenAIEmbeddings 초기화 완료")
print("   모델: text-embedding-3-small")

# 1. 단일 텍스트 임베딩
print("\n[1/3] 단일 텍스트 임베딩")
text = "청크 임베딩 테스트"
vec = chunkie_openai_embeddins.embed(text)
print(f"   텍스트: '{text}'")
print(f"   임베딩 차원: {len(vec)}")
print(f"   임베딩 벡터 (처음 5개): {vec[:5]}")

# 2. 배치 임베딩
print("\n[2/3] 배치 임베딩")
texts = ["첫 번째 문장입니다.", "두 번째 문장입니다.", "세 번째 문장입니다."]
vecs = chunkie_openai_embeddins.embed_batch(texts)
print(f"   입력 텍스트 개수: {len(texts)}")
print(f"   출력 벡터 개수: {len(vecs)}")
print(f"   각 벡터 차원: {len(vecs[0])}")

 OpenAIEmbeddings 초기화 완료
   모델: text-embedding-3-small

[1/3] 단일 텍스트 임베딩
   텍스트: '청크 임베딩 테스트'
   임베딩 차원: 1536
   임베딩 벡터 (처음 5개): [-0.03322652 -0.02554174 -0.00478171 -0.04074111 -0.02861827]

[2/3] 배치 임베딩
   입력 텍스트 개수: 3
   출력 벡터 개수: 3
   각 벡터 차원: 1536


## 2. TEI(Hugging Face Text Embeddings Inference) 기반 Chunkie CustomEmbeddings

TEI는 Hugging Face가 제공하는 고성능 임베딩 추론 서버입니다.  
Chonkie의 CustomEmbeddings를 통해 TEI 서버와 연동할 수 있습니다.
  
**TEI 서버 시작 (GPU 가 있는 Docker):**
```bash
docker run --rm -p 8080:80 ghcr.io/huggingface/text-embeddings-inference:1.8 \
  --model-id Qwen/Qwen3-Embedding-0.6B
```

**특징:**
- GPU 가속 지원, On-Premise 배포 가능 (데이터 보안)
- 다양한 오픈소스 임베딩 모델 지원, REST API 기반
- [**Docker Image 버전에 따라서 지원하는 GPU 가 다르니 꼭 Github 에서 확인해주세요.**](https://github.com/huggingface/text-embeddings-inference?tab=readme-ov-file#docker-images)

### Docker Images

Text Embeddings Inference ships with multiple Docker images that you can use to target a specific backend:

| Architecture                        | Image                                                                   |
|-------------------------------------|-------------------------------------------------------------------------|
| CPU                                 | ghcr.io/huggingface/text-embeddings-inference:cpu-1.8                   |
| Volta                               | NOT SUPPORTED                                                           |
| Turing (T4, RTX 2000 series, ...)   | ghcr.io/huggingface/text-embeddings-inference:turing-1.8 (experimental) |
| Ampere 80 (A100, A30)               | ghcr.io/huggingface/text-embeddings-inference:1.8                       |
| Ampere 86 (A10, A40, ...)           | ghcr.io/huggingface/text-embeddings-inference:86-1.8                    |
| Ada Lovelace (RTX 4000 series, ...) | ghcr.io/huggingface/text-embeddings-inference:89-1.8                    |
| Hopper (H100)                       | ghcr.io/huggingface/text-embeddings-inference:hopper-1.8 (experimental) |

**Warning**: Flash Attention is turned off by default for the Turing image as it suffers from precision issues.
You can turn Flash Attention v1 ON by using the `USE_FLASH_ATTENTION=True` environment variable.

### API documentation

You can consult the OpenAPI documentation of the `text-embeddings-inference` REST API using the `/docs` route.
The Swagger UI is also available
at: [https://huggingface.github.io/text-embeddings-inference](https://huggingface.github.io/text-embeddings-inference).


### **참고 문서:**
- [TEI GitHub](https://github.com/huggingface/text-embeddings-inference)
- [TEI Docs](https://huggingface.github.io/text-embeddings-inference/)
- [CustomEmbeddings 문서](https://docs.chonkie.ai/oss/embeddings/custom-embeddings)


In [35]:
# ============================================================================
# TEI CustomEmbeddings 구현
# ============================================================================
import os

import httpx
import numpy as np
from chonkie.embeddings import BaseEmbeddings

# TEI 서버 URL 설정
RUNPOD_TEI_SERVER = "https://dew21ae8bkg6ce-8000.proxy.runpod.net"
TEI_BASE_URL = os.getenv("TEI_BASE_URL", RUNPOD_TEI_SERVER)


from chonkie.tokenizer import Tokenizer


class TEITokenizer(Tokenizer):
    """TEI 서버를 위한 Tokenizer wrapper

    Chonkie의 Tokenizer 클래스를 상속받아 AutoTokenizer가 "chonkie" backend로 인식합니다.
    """

    # AutoTokenizer가 "chonkie" backend로 인식하도록 __module__ 설정
    __module__ = "chonkie.tokenizer"

    def __init__(self, tei_embeddings):
        """TEIEmbeddings 인스턴스를 받아서 래핑합니다."""
        # Tokenizer의 __init__을 호출하지 않음 (vocab 불필요)
        self._embeddings = tei_embeddings

    def __repr__(self) -> str:
        """문자열 표현"""
        return f"TEITokenizer(base_url={self._embeddings.base_url})"

    def encode(self, text: str) -> list[int]:
        """텍스트를 토큰 ID로 인코딩"""
        return self._embeddings.encode(text)

    def decode(self, tokens: list[int]) -> str:
        """토큰 ID를 텍스트로 디코딩"""
        return self._embeddings.decode(tokens)

    def tokenize(self, text: str) -> list[int]:
        """텍스트를 토큰화 (encode와 동일)"""
        return self._embeddings.encode(text)

    def encode_batch(self, texts: list[str]) -> list[list[int]]:
        """배치 인코딩"""
        return self._embeddings.encode_batch(texts)

    def decode_batch(self, token_sequences: list[list[int]]) -> list[str]:
        """배치 디코딩"""
        return self._embeddings.decode_batch(token_sequences)

    def count_tokens(self, text: str) -> int:
        """토큰 수 카운트"""
        return self._embeddings.count_tokens(text)

    def count_tokens_batch(self, texts: list[str]) -> list[int]:
        """배치 토큰 카운트"""
        return self._embeddings.count_tokens_batch(texts)


class TEIEmbeddings(BaseEmbeddings):
    """TEI(Text Embeddings Inference) 서버를 위한 Chonkie CustomEmbeddings 구현"""

    def __init__(self, base_url: str):
        self.base_url = base_url
        self._cached_dim = None
        self._cached_info = None
        self._client = httpx.Client(timeout=60.0, verify=False)
        self._tokenizer = None  # lazy init

    def __del__(self):
        """리소스 정리"""
        if hasattr(self, "_client"):
            self._client.close()

    @property
    def dimension(self) -> int:
        """임베딩 차원 반환 (최초 임베딩 시 캐싱)"""
        if self._cached_dim is None:
            # 더미 텍스트로 차원 확인
            self._cached_dim = len(self.embed("dimension check"))
        return self._cached_dim

    def embed(self, text: str) -> np.ndarray:
        """단일 텍스트 임베딩"""
        resp = self._client.post(f"{self.base_url}/embed", json={"inputs": text}, timeout=30.0)
        resp.raise_for_status()
        return np.array(resp.json()[0])  # TEI는 배열로 반환

    def embed_batch(self, texts: list[str]) -> list[np.ndarray]:
        """배치 텍스트 임베딩"""
        resp = self._client.post(f"{self.base_url}/embed", json={"inputs": texts}, timeout=60.0)
        resp.raise_for_status()
        return [np.array(vec) for vec in resp.json()]

    def count_tokens(self, text: str) -> int:
        """토큰 수 측정 (TEI /tokenize 엔드포인트 사용)"""
        if not text or not text.strip():
            return 0

        try:
            resp = self._client.post(
                f"{self.base_url}/tokenize", json={"inputs": text}, timeout=30.0
            )
            resp.raise_for_status()
            # 응답 형식: List[List[TokenizeResponse]]
            # TokenizeResponse: {"id": int, "text": str, "start": int, "stop": int}
            result = resp.json()
            if result and len(result) > 0:
                token_count = len(result[0])
                # 명시적으로 int로 변환 (numpy int 등을 방지)
                return int(token_count)
            return 0
        except Exception as e:
            print(f"Warning: count_tokens failed for text (len={len(text)}): {e}")
            # Fallback: encode를 사용
            return len(self.encode(text))

    def count_tokens_batch(self, texts: list[str]) -> list[int]:
        """배치 토큰 수 측정 (각 텍스트를 순차적으로 /tokenize에 요청)"""
        return [self.count_tokens(text) for text in texts]

    def get_tokenizer(self):
        """토크나이저 객체 반환 (TEITokenizer wrapper 반환)"""
        if self._tokenizer is None:
            self._tokenizer = TEITokenizer(self)
        return self._tokenizer

    def encode(self, text: str, add_special_tokens: bool = True) -> list[int]:
        """텍스트를 토큰 ID로 인코딩 (TEI /tokenize 엔드포인트 사용)

        Args:
            text: 인코딩할 텍스트
            add_special_tokens: 특수 토큰 추가 여부 (TEI가 자동 처리)

        Returns:
            토큰 ID 리스트
        """
        resp = self._client.post(f"{self.base_url}/tokenize", json={"inputs": text}, timeout=30.0)
        resp.raise_for_status()
        # TEI 응답: List[List[TokenizeResponse]]
        # TokenizeResponse: {"id": int, "text": str, "start": int, "stop": int}
        token_data = resp.json()[0]
        token_ids = [token["id"] for token in token_data]
        return token_ids

    def tokenize(self, text: str) -> list[int]:
        """텍스트를 토큰화 (encode와 동일)

        Args:
            text: 토큰화할 텍스트

        Returns:
            토큰 ID 리스트
        """
        return self.encode(text)

    def encode_batch(self, texts: list[str]) -> list[list[int]]:
        """배치 텍스트를 토큰 ID로 인코딩

        Args:
            texts: 인코딩할 텍스트 리스트

        Returns:
            토큰 ID 리스트의 리스트
        """
        return [self.encode(text) for text in texts]

    def decode_batch(self, token_sequences: list[list[int]]) -> list[str]:
        """배치 토큰 ID를 텍스트로 디코딩

        Args:
            token_sequences: 디코딩할 토큰 ID 리스트의 리스트

        Returns:
            디코딩된 텍스트 리스트
        """
        return [self.decode(tokens) for tokens in token_sequences]

    def decode(self, token_ids: list[int], skip_special_tokens: bool = True) -> str:
        """토큰 ID를 텍스트로 디코딩

        Args:
            token_ids: 디코딩할 토큰 ID 리스트
            skip_special_tokens: 특수 토큰 제거 여부

        Returns:
            디코딩된 텍스트
        """
        # Lazy loading: 첫 호출 시 로컬 토크나이저 로드
        if not hasattr(self, "_local_tokenizer"):
            try:
                from tokenizers import Tokenizer

                info = self.get_info()
                model_id = info.get("model_id", "Qwen/Qwen3-Embedding-4B")
                self._local_tokenizer = Tokenizer.from_pretrained(model_id)
            except Exception as e:
                print(f"로컬 토크나이저 로드 실패: {e}. 'pip install tokenizers'로 설치 필요")
                raise

        return self._local_tokenizer.decode(token_ids, skip_special_tokens=skip_special_tokens)

    @classmethod
    def is_available(cls) -> bool:
        """TEI 서버 가용성 확인"""
        try:
            with httpx.Client(timeout=5.0, verify=False) as client:
                r = client.get(f"{TEI_BASE_URL}/health")
                return r.status_code == 200
        except Exception:
            return False

    def get_info(self) -> dict:
        """TEI 서버의 모델 정보 조회 (/info 엔드포인트 사용)"""
        if self._cached_info is None:
            try:
                resp = self._client.get(f"{self.base_url}/info", timeout=10.0)
                resp.raise_for_status()
                self._cached_info = resp.json()
            except Exception:
                self._cached_info = {}
        return self._cached_info

    def __repr__(self) -> str:
        """모델 정보를 포함한 문자열 표현"""
        try:
            info = self.get_info()
            model_id = info.get("model_id", "unknown")
            version = info.get("version", "unknown")
            return f"TEIEmbeddings(base_url='{self.base_url}', model='{model_id}', version='{version}')"
        except Exception:
            return f"TEIEmbeddings(base_url='{self.base_url}')"


# TEI 서버 가용성 확인
print("\n[1/4] TEI 서버 가용성 확인")
print(f"   TEI URL: {TEI_BASE_URL}")
if TEIEmbeddings.is_available():
    print("    TEI 서버 연결 성공!")

# TEIEmbeddings 인스턴스 생성
print("\n[2/4] TEIEmbeddings 초기화")
chunkie_tei_embeddings = TEIEmbeddings(base_url=TEI_BASE_URL)
print(f"   {chunkie_tei_embeddings}")

# 단일 임베딩 테스트
print("\n[3/4] 단일 임베딩 테스트")
vec = chunkie_tei_embeddings.embed("Huggingface TEI 연동 테스트")
print(f"   임베딩 차원: {len(vec)}")
print(f"   임베딩 벡터 (처음 5개): {vec[:5]}")

# 배치 임베딩 테스트
print("\n[4/4] 배치 임베딩 테스트")
texts = ["첫 번째 문장", "두 번째 문장", "세 번째 문장"]
vecs = chunkie_tei_embeddings.embed_batch(texts)
print(f"   입력 개수: {len(texts)}")
print(f"   출력 개수: {len(vecs)}")
print(f"   각 벡터 차원: {len(vecs[0])}")

# Tokenizer 기능 테스트
print("\n[추가] Tokenizer 기능 테스트")
print("=" * 60)

# 1. get_tokenizer() 테스트
print("\n[1] get_tokenizer() 테스트")
tokenizer = chunkie_tei_embeddings.get_tokenizer()
print(f"   토크나이저 타입: {type(tokenizer)}")
print(f"   토크나이저 == self: {tokenizer is chunkie_tei_embeddings}")

# 2. encode() 테스트
print("\n[2] encode() 테스트")
test_text = "안녕하세요, Chonkie 테스트입니다."
token_ids = chunkie_tei_embeddings.encode(test_text)
print(f"   입력 텍스트: {test_text}")
print(f"   토큰 ID 개수: {len(token_ids)}")
print(f"   토큰 ID (처음 10개): {token_ids[:10]}")

# 3. decode() 테스트
print("\n[3] decode() 테스트")
decoded_text = chunkie_tei_embeddings.decode(token_ids)
print(f"   원본 텍스트: {test_text}")
print(f"   디코딩 텍스트: {decoded_text}")
print(f"   일치 여부: {test_text.strip() == decoded_text.strip()}")

# 4. Tokenizer Protocol 호환성 확인
print("\n[4] Tokenizer Protocol 호환성")
print(f"   hasattr(tokenizer, 'encode'): {hasattr(tokenizer, 'encode')}")
print(f"   hasattr(tokenizer, 'decode'): {hasattr(tokenizer, 'decode')}")
print(f"   callable(tokenizer.encode): {callable(tokenizer.encode)}")
print(f"   callable(tokenizer.decode): {callable(tokenizer.decode)}")

print("\nTokenizer 구현 완료!")



[1/4] TEI 서버 가용성 확인
   TEI URL: https://dew21ae8bkg6ce-8000.proxy.runpod.net
    TEI 서버 연결 성공!

[2/4] TEIEmbeddings 초기화
   TEIEmbeddings(base_url='https://dew21ae8bkg6ce-8000.proxy.runpod.net', model='Qwen/Qwen3-Embedding-4B', version='1.8.3')

[3/4] 단일 임베딩 테스트
   임베딩 차원: 2560
   임베딩 벡터 (처음 5개): [-0.0002205   0.01461212  0.01620024  0.0285861  -0.00088334]

[4/4] 배치 임베딩 테스트
   입력 개수: 3
   출력 개수: 3
   각 벡터 차원: 2560

[추가] Tokenizer 기능 테스트

[1] get_tokenizer() 테스트
   토크나이저 타입: <class 'chonkie.tokenizer.TEITokenizer'>
   토크나이저 == self: False

[2] encode() 테스트
   입력 텍스트: 안녕하세요, Chonkie 테스트입니다.
   토큰 ID 개수: 13
   토큰 ID (처음 10개): [126246, 144370, 91145, 11, 910, 263, 30322, 10764, 72509, 53189]

[3] decode() 테스트
   원본 텍스트: 안녕하세요, Chonkie 테스트입니다.
   디코딩 텍스트: 안녕하세요, Chonkie 테스트입니다.
   일치 여부: True

[4] Tokenizer Protocol 호환성
   hasattr(tokenizer, 'encode'): True
   hasattr(tokenizer, 'decode'): True
   callable(tokenizer.encode): True
   callable(tokenizer.decode): True

Tokenizer 구현 완료!


In [ ]:
# ===========================================================================================
# TEIEmbeddings와 Chonkie Chunker 통합 테스트
# ===========================================================================================
print("\n" + "=" * 80)
print("TEIEmbeddings와 Chonkie Chunker 통합 테스트")
print("=" * 80)

# 디버깅: TEITokenizer와 AutoTokenizer 확인
print("\n[DEBUG] TEITokenizer와 AutoTokenizer 확인")
from chonkie.tokenizer import AutoTokenizer

# get_tokenizer()로 TEITokenizer 가져오기
tei_tokenizer = chunkie_tei_embeddings.get_tokenizer()
print(f"   TEITokenizer 타입: {type(tei_tokenizer)}")
print(f"   TEITokenizer 클래스명: {tei_tokenizer.__class__.__name__}")
print(f"   'chonkie' in type string: {'chonkie' in str(type(tei_tokenizer)).lower()}")

# AutoTokenizer로 래핑
test_auto_tokenizer = AutoTokenizer(tei_tokenizer)
print(f"\n   AutoTokenizer Backend: {test_auto_tokenizer._backend}")
print(f"   Wrapped tokenizer type: {type(test_auto_tokenizer.tokenizer)}")

# 간단한 토큰 카운트 테스트
test_text = "테스트"
try:
    count = test_auto_tokenizer.count_tokens(test_text)
    print(f"\n   count_tokens('{test_text}'): {count}")
    print(f"   타입: {type(count)}")
    print(f"   isinstance(count, int): {isinstance(count, int)}")
    print(f"   count >= 0: {count >= 0}")
    print("    count_tokens 성공!")
except Exception as e:
    print(f"     count_tokens 오류: {e}")
    import traceback

    traceback.print_exc()

print()

# 샘플 텍스트 (한국어)
sample_text = """
인공지능 기술의 발전은 우리 삶의 많은 부분을 변화시키고 있습니다.
특히 자연어 처리 분야에서는 놀라운 성과를 보이고 있습니다.

텍스트 임베딩은 자연어 처리의 핵심 기술 중 하나입니다.
문장이나 문서를 고차원 벡터 공간에 매핑하여 의미를 수치화합니다.
이를 통해 컴퓨터가 텍스트의 의미를 이해하고 처리할 수 있게 됩니다.

청킹(Chunking)은 긴 텍스트를 작은 단위로 나누는 과정입니다.
검색 증강 생성(RAG) 시스템에서 매우 중요한 전처리 단계입니다.
적절한 청킹 전략은 검색 품질과 생성 결과에 큰 영향을 미칩니다.

의미 기반 청킹은 텍스트의 의미적 일관성을 고려합니다.
단순히 길이만 고려하는 것이 아니라 내용의 연관성을 판단합니다.
이를 위해 임베딩 모델이 필수적으로 사용됩니다.
"""

# 1. TokenChunker와 TEIEmbeddings
print("\n[1] TokenChunker + TEIEmbeddings")
try:
    from chonkie.chunker import TokenChunker

    token_chunker = TokenChunker(
        tokenizer=chunkie_tei_embeddings.get_tokenizer(), chunk_size=100, chunk_overlap=20
    )

    token_chunks = token_chunker.chunk(sample_text)
    print(f"   생성된 청크 수: {len(token_chunks)}")
    print(
        f"   첫 번째 청크 길이: {len(token_chunks[0].text) if hasattr(token_chunks[0], 'text') else len(str(token_chunks[0]))} 문자"
    )
    print(
        f"   첫 번째 청크 미리보기: {(token_chunks[0].text if hasattr(token_chunks[0], 'text') else str(token_chunks[0]))[:100]}..."
    )
    print("    TokenChunker 통합 성공!")
except Exception as e:
    print(f"     오류: {e}")

# 2. SemanticChunker와 TEIEmbeddings
print("\n[2] SemanticChunker + TEIEmbeddings")
try:
    from chonkie.chunker import SemanticChunker

    semantic_chunker = SemanticChunker(
        embedding_model=chunkie_tei_embeddings, chunk_size=200, threshold=0.5
    )

    semantic_chunks = semantic_chunker.chunk(sample_text)
    print(f"   생성된 청크 수: {len(semantic_chunks)}")
    print(
        f"   첫 번째 청크 길이: {len(semantic_chunks[0].text) if hasattr(semantic_chunks[0], 'text') else len(str(semantic_chunks[0]))} 문자"
    )
    print(
        f"   첫 번째 청크 미리보기: {(semantic_chunks[0].text if hasattr(semantic_chunks[0], 'text') else str(semantic_chunks[0]))[:100]}..."
    )
    print("    SemanticChunker 통합 성공!")
except Exception as e:
    print(f"     오류: {e}")

# 3. SentenceChunker와 TEIEmbeddings
print("\n[3] SentenceChunker + TEIEmbeddings")
try:
    from chonkie.chunker import SentenceChunker

    sentence_chunker = SentenceChunker(
        tokenizer=chunkie_tei_embeddings.get_tokenizer(), chunk_size=150, chunk_overlap=10
    )

    sentence_chunks = sentence_chunker.chunk(sample_text)
    print(f"   생성된 청크 수: {len(sentence_chunks)}")
    print(
        f"   첫 번째 청크 길이: {len(sentence_chunks[0].text) if hasattr(sentence_chunks[0], 'text') else len(str(sentence_chunks[0]))} 문자"
    )
    print(
        f"   첫 번째 청크 미리보기: {(sentence_chunks[0].text if hasattr(sentence_chunks[0], 'text') else str(sentence_chunks[0]))[:100]}..."
    )
    print("    SentenceChunker 통합 성공!")
except Exception as e:
    print(f"     오류: {e}")

print("\n" + "=" * 80)
print(" TEIEmbeddings와 Chonkie Chunker 통합 테스트 완료!")
print("=" * 80)



TEIEmbeddings와 Chonkie Chunker 통합 테스트

[DEBUG] TEITokenizer와 AutoTokenizer 확인
   TEITokenizer 타입: <class 'chonkie.tokenizer.TEITokenizer'>
   TEITokenizer 클래스명: TEITokenizer
   'chonkie' in type string: True

   AutoTokenizer Backend: chonkie
   Wrapped tokenizer type: <class 'chonkie.tokenizer.TEITokenizer'>


2025-11-10 17:56:30.211 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized TokenChunker
2025-11-10 17:56:30.212 | DEBUG    | chonkie.chunker.token:chunk:128 - Chunking text of length 392 with chunk_size=100



   count_tokens('테스트'): 3
   타입: <class 'int'>
   isinstance(count, int): True
   count >= 0: True
   ✅ count_tokens 성공!


[1] TokenChunker + TEIEmbeddings


2025-11-10 17:56:30.563 | INFO     | chonkie.chunker.token:chunk:143 - Created 3 chunks from 237 tokens
2025-11-10 17:56:30.564 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized SemanticChunker
2025-11-10 17:56:30.564 | DEBUG    | chonkie.chunker.semantic:chunk:531 - Starting semantic chunking for text of length 392


   생성된 청크 수: 3
   첫 번째 청크 길이: 170 문자
   첫 번째 청크 미리보기: 
인공지능 기술의 발전은 우리 삶의 많은 부분을 변화시키고 있습니다.
특히 자연어 처리 분야에서는 놀라운 성과를 보이고 있습니다.

텍스트 임베딩은 자연어 처리의 핵심 기술 중 하...
   ✅ TokenChunker 통합 성공!

[2] SemanticChunker + TEIEmbeddings


2025-11-10 17:56:34.156 | DEBUG    | chonkie.chunker.semantic:chunk:535 - Prepared 11 sentences for semantic analysis
2025-11-10 17:56:36.028 | INFO     | chonkie.chunker.semantic:chunk:573 - Created 3 semantic chunks from 11 sentences
2025-11-10 17:56:36.029 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized SentenceChunker
2025-11-10 17:56:36.038 | DEBUG    | chonkie.chunker.sentence:chunk:304 - Chunking text of length 392


   생성된 청크 수: 3
   첫 번째 청크 길이: 222 문자
   첫 번째 청크 미리보기: 
인공지능 기술의 발전은 우리 삶의 많은 부분을 변화시키고 있습니다.
특히 자연어 처리 분야에서는 놀라운 성과를 보이고 있습니다.

텍스트 임베딩은 자연어 처리의 핵심 기술 중 하...
   ✅ SemanticChunker 통합 성공!

[3] SentenceChunker + TEIEmbeddings


2025-11-10 17:56:39.625 | DEBUG    | chonkie.chunker.sentence:chunk:312 - Prepared 11 sentences for chunking
2025-11-10 17:56:40.282 | INFO     | chonkie.chunker.sentence:chunk:386 - Created 2 chunks from text


   생성된 청크 수: 2
   첫 번째 청크 길이: 222 문자
   첫 번째 청크 미리보기: 
인공지능 기술의 발전은 우리 삶의 많은 부분을 변화시키고 있습니다.
특히 자연어 처리 분야에서는 놀라운 성과를 보이고 있습니다.

텍스트 임베딩은 자연어 처리의 핵심 기술 중 하...
   ✅ SentenceChunker 통합 성공!

✅ TEIEmbeddings와 Chonkie Chunker 통합 테스트 완료!


## 3. Chunkie 에서 제공하는 Chunkers

다양한 청킹 전략을 비교합니다.

**참고 문서:**
- [Chunkers 개요](https://docs.chonkie.ai/oss/chunkers/overview)


In [ ]:
# ===========================================================================================
# Chunkers - TokenChunker, SentenceChunker, RecursiveChunker, SemanticChunker, SlumberChunker
# ===========================================================================================
import os

from chonkie.chunker import (
    RecursiveChunker,
    SemanticChunker,
    SentenceChunker,
    SlumberChunker,
    TokenChunker,
)
from chonkie.genie import OpenAIGenie

# 샘플 텍스트 준비
sample_text = """Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.

첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.
빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째로 SentenceChunker는 문장 단위로 분할합니다.
자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.

세 번째로 RecursiveChunker는 계층적으로 분할합니다.
문단, 문장, 단어 순서로 구조를 보존합니다.

네 번째로는 SemanticChunker는 의미 유사도 기반으로 분할합니다.
가장 정교하지만 임베딩 모델이 필요합니다.

마지막으로는 SlumberChunker는 LLM 기반 Contextual Chunking 입니다."""

print(f"\n샘플 텍스트 길이: {len(sample_text)} 문자\n")
# 1) TokenChunker
print("[1/4] TokenChunker (고정 토큰)")
chunker_token = TokenChunker(chunk_size=64, chunk_overlap=10)
chunks_token = chunker_token.chunk(sample_text)
print(f"   생성된 청크 수: {len(chunks_token)}")
print(f"   첫 청크: {chunks_token[0].text[:80]}...")

# 2) SentenceChunker
print("\n[2/4] SentenceChunker (문장 단위)")
chunker_sent = SentenceChunker(chunk_size=128, min_sentences_per_chunk=2)
chunks_sent = chunker_sent.chunk(sample_text)
print(f"   생성된 청크 수: {len(chunks_sent)}")
print(f"   첫 청크: {chunks_sent[0].text[:80]}...")

# 3) RecursiveChunker
print("\n[3/4] RecursiveChunker (재귀적 분할)")
chunker_recur = RecursiveChunker(chunk_size=128, min_characters_per_chunk=24)
chunks_recur = chunker_recur.chunk(sample_text)
print(f"   생성된 청크 수: {len(chunks_recur)}")
print(f"   첫 청크: {chunks_recur[0].text[:80]}...")

# 4) SemanticChunker (임베딩 필요)
print("\n[4/4] SemanticChunker (의미 유사도 기반)")
chunker_sem = SemanticChunker(
    embedding_model=chunkie_openai_embeddins,
    # embedding_model=chunkie_tei_embeddings,
    chunk_size=1000,
    threshold=0.5,
)
chunks_sem = chunker_sem.chunk(sample_text)
print(f"   생성된 청크 수: {len(chunks_sem)}")
print(f"   첫 청크: {chunks_sem[0].text}...")

# 5) SlumberChunker (LLM 기반 Contextual Chunking)
print("\n[5/5] SlumberChunker (LLM 기반 Contextual Chunking)")
chunker_slumber = SlumberChunker(
    genie=OpenAIGenie(
        model="openai/gpt-4.1",
        base_url=os.getenv("OPENROUTER_BASE_URL"),
        api_key=os.getenv("OPENROUTER_API_KEY"),
    ),
    chunk_size=1200,
    candidate_size=200,
)
chunks_slumber = chunker_slumber.chunk(sample_text)
print(f"   생성된 청크 수: {len(chunks_slumber)}")
print(f"   첫 청크: {chunks_slumber[0].text[:80]}...")

# 청크 수 비교
print("청크 수 비교:")
print(f"  TokenChunker:     {len(chunks_token)}개")
print(f"  SentenceChunker:  {len(chunks_sent)}개")
print(f"  RecursiveChunker: {len(chunks_recur)}개")
print(f"  SemanticChunker:  {len(chunks_sem)}개")
print(f"  SlumberChunker:    {len(chunks_slumber)}개")

2025-11-10 17:42:06.655 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized TokenChunker
2025-11-10 17:42:06.655 | DEBUG    | chonkie.chunker.token:chunk:128 - Chunking text of length 411 with chunk_size=64
2025-11-10 17:42:06.656 | INFO     | chonkie.chunker.token:chunk:143 - Created 8 chunks from 411 tokens
2025-11-10 17:42:06.657 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized SentenceChunker
2025-11-10 17:42:06.657 | DEBUG    | chonkie.chunker.sentence:chunk:304 - Chunking text of length 411
2025-11-10 17:42:06.658 | DEBUG    | chonkie.chunker.sentence:chunk:312 - Prepared 12 sentences for chunking
2025-11-10 17:42:06.659 | INFO     | chonkie.chunker.sentence:chunk:386 - Created 4 chunks from text
2025-11-10 17:42:06.659 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized RecursiveChunker
2025-11-10 17:42:06.660 | DEBUG    | chonkie.chunker.recursive:chunk:368 - Starting recursive chunking for text of length 411
2025-11-10 17:42:06.660 | INFO     | chonk


샘플 텍스트 길이: 411 문자

[1/4] TokenChunker (고정 토큰)
   생성된 청크 수: 8
   첫 청크: Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여...

[2/4] SentenceChunker (문장 단위)
   생성된 청크 수: 4
   첫 청크: Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다...

[3/4] RecursiveChunker (재귀적 분할)
   생성된 청크 수: 4
   첫 청크: Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다...

[4/4] SemanticChunker (의미 유사도 기반)


2025-11-10 17:42:08.099 | INFO     | chonkie.chunker.semantic:chunk:573 - Created 3 semantic chunks from 11 sentences
2025-11-10 17:42:08.104 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized SlumberChunker
2025-11-10 17:42:08.105 | DEBUG    | chonkie.chunker.slumber:chunk:249 - Starting slumber chunking for text of length 411
2025-11-10 17:42:08.105 | DEBUG    | chonkie.chunker.slumber:chunk:251 - Created 11 initial splits for LLM-based semantic boundary detection


   생성된 청크 수: 3
   첫 청크: Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.

첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.
빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째로 SentenceChunker는 문장 단위로 분할합니다.
자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.
...

[5/5] SlumberChunker (LLM 기반 Contextual Chunking)


🦛 choooooooooooooooooooonk 100% • 11/11 splits processed [00:03<00:00,  2.99split/s] 🌱2025-11-10 17:42:11.476 | INFO     | chonkie.chunker.slumber:chunk:304 - Created 2 chunks using LLM-guided semantic splitting
🦛 choooooooooooooooooooonk 100% • 11/11 splits processed [00:03<00:00,  3.26split/s] 🌱

   생성된 청크 수: 2
   첫 청크: Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다...
청크 수 비교:
  TokenChunker:     8개
  SentenceChunker:  4개
  RecursiveChunker: 4개
  SemanticChunker:  3개
  SlumberChunker:    2개


### Experimental: CodeChunker

코드 전용 청킹 전략입니다. 프로그래밍 언어의 구조를 인식하여 분할합니다.

**참고 문서:**
- [Code Chunker](https://docs.chonkie.ai/oss/experimental/code-chunker)


In [ ]:
# ============================================================================
# CodeChunker - Experimental
# CodeChunker는 함수/클래스 단위로 분할하여 코드 구조를 보존합니다.
# ============================================================================
try:
    from chonkie.chunker import CodeChunker

    # Python 코드 샘플
    code_text = '''
# utils.py
def add(a, b):
    """두 수를 더합니다."""
    return a + b

def subtract(a, b):
    """두 수를 뺍니다."""
    return a - b

class Calculator:
    """계산기 클래스"""

    def __init__(self):
        self.result = 0

    def multiply(self, a, b):
        """두 수를 곱합니다."""
        self.result = a * b
        return self.result

    def divide(self, a, b):
        """두 수를 나눕니다."""
        if b == 0:
            raise ValueError("0으로 나눌 수 없습니다")
        self.result = a / b
        return self.result
'''

    print(f"\n코드 샘플 길이: {len(code_text)} 문자\n")

    # CodeChunker로 분할
    print("[1/1] CodeChunker (Python)")
    chunker_code = CodeChunker(language="python", chunk_size=200)
    chunks_code = chunker_code.chunk(code_text)

    print(f"   생성된 청크 수: {len(chunks_code)}")
    for i, chunk in enumerate(chunks_code):
        print(f"\n   --- 청크 {i + 1} ---")
        print(f"   {chunk.text}")

except ImportError:
    print("  pip install 'chonkie[code]' 또는 'chonkie[all]'로 설치하세요.")
except Exception as e:
    print(f"오류: {e}")


코드 샘플 길이: 494 문자

[1/1] CodeChunker (Python)


2025-11-09 17:27:50.519 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized CodeChunker
2025-11-09 17:27:51.121 | DEBUG    | chonkie.chunker.code:chunk:328 - Starting code chunking for text of length 494
2025-11-09 17:27:51.122 | DEBUG    | chonkie.chunker.code:chunk:339 - Using configured language: python
2025-11-09 17:27:51.123 | INFO     | chonkie.chunker.code:chunk:356 - Created 3 code chunks from parsed syntax tree


   생성된 청크 수: 3

   --- 청크 1 ---
   
# utils.py
def add(a, b):
    """두 수를 더합니다."""
    return a + b

def subtract(a, b):
    """두 수를 뺍니다."""
    return a - b

...

   --- 청크 2 ---
   class Calculator:
    """계산기 클래스"""

    def __init__(self):
        self.result = 0

    def multiply(self, a, b):
        """두 수를 곱합니다."""
        s...

   --- 청크 3 ---
   def divide(self, a, b):
        """두 수를 나눕니다."""
        if b == 0:
            raise ValueError("0으로 나눌 수 없습니다")
        self.result = a / b
        ...

CodeChunker는 함수/클래스 단위로 분할하여 코드 구조를 보존합니다.


## 4. Refinery (문맥 보강)

청크에 `추가 문맥 정보`를 부착하여 검색 품질을 향상시킬 수 있습니다.

**종류:**
- **OverlapRefinery**: 인접 청크의 내용을 오버랩으로 추가
- **EmbeddingsRefinery**: 임베딩 벡터를 청크에 부착

**참고 문서:**
- [Refinery 개요](https://docs.chonkie.ai/oss/refinery/overview)


### Refinery 상세 설명

#### EmbeddingsRefinery

**클래스 개요**

`EmbeddingsRefinery`는 청크의 텍스트를 임베딩 모델을 사용하여 임베딩하고, 청크에 임베딩 벡터를 부착합니다. 이는 벡터 데이터베이스에 upsert하는 등의 다운스트림 작업에 사용됩니다.

**초기화 (`__init__`)**

```python
def __init__(
    self,
    embedding_model: Union[str, BaseEmbeddings, AutoEmbeddings] = "minishlab/potion-retrieval-32M",
    **kwargs: Dict[str, Any],
) -> None:
```

**파라미터:**
- `embedding_model`: 사용할 임베딩 모델
  - 타입: `Union[str, BaseEmbeddings, AutoEmbeddings]`
  - 기본값: `"minishlab/potion-retrieval-32M"`
  - 설명: 문자열로 모델 이름을 전달하거나, `BaseEmbeddings` 인스턴스를 직접 전달할 수 있습니다. 문자열인 경우 `AutoEmbeddings`를 통해 자동으로 로드됩니다.

**리파인 메서드 (`refine`)**

```python
def refine(self, chunks: List[Chunk]) -> List[Chunk]:
    """Refine the chunks."""
    logger.debug(f"Starting embedding refinery for {len(chunks)} chunks")
    texts = [chunk.text for chunk in chunks]
    embeds = self.embedding_model.embed_batch(texts)
    for chunk, embed in zip(chunks, embeds):
        chunk.embedding = embed
    logger.info(f"Embedding refinement complete: added embeddings to {len(chunks)} chunks")
    return chunks
```

**동작 방식:**
1. 입력된 청크 리스트에서 텍스트를 추출합니다.
2. `embed_batch()` 메서드를 사용하여 배치로 임베딩을 생성합니다 (성능 최적화).
3. 각 청크에 해당하는 임베딩 벡터를 `chunk.embedding` 속성에 부착합니다.
4. 임베딩이 부착된 청크 리스트를 반환합니다.

**사용 예시:**

```python
from chonkie.refinery import EmbeddingsRefinery
from chonkie.embeddings import OpenAIEmbeddings

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# EmbeddingsRefinery 초기화
refinery = EmbeddingsRefinery(embedding_model=embeddings)

# 청크에 임베딩 부착
refined_chunks = refinery.refine(chunks)
```

---

#### OverlapRefinery

**클래스 개요**

`OverlapRefinery`는 청크에 오버랩을 추가하여 문맥을 보강합니다. LRU 캐싱(maxsize=8192)을 사용하여 토크나이제이션 작업의 성능을 향상시킵니다. 캐시는 `cache_info()`로 모니터링하고 `clear_cache()`로 초기화할 수 있습니다.

**초기화 (`__init__`)**

```python
def __init__(
    self,
    tokenizer: Union[str, TokenizerProtocol] = "character",
    context_size: Union[int, float] = 0.25,
    mode: Literal["token", "recursive"] = "token",
    method: Literal["suffix", "prefix"] = "suffix",
    rules: RecursiveRules = RecursiveRules(),
    merge: bool = True,
    inplace: bool = True,
) -> None:
```

**파라미터:**

| 파라미터 | 타입 | 기본값 | 설명 |
|---------|------|-------|------|
| `tokenizer` | `Union[str, TokenizerProtocol]` | `"character"` | 사용할 토크나이저. 제공되지 않으면 문자 단위 오버랩을 기본으로 사용합니다. |
| `context_size` | `Union[int, float]` | `0.25` | 청크에 추가할 컨텍스트 크기. 정수인 경우 토큰 수, float인 경우 최대 청크 크기의 비율 (0-1 사이). |
| `mode` | `Literal["token", "recursive"]` | `"token"` | 오버랩 모드. `"token"`은 토큰 기반, `"recursive"`는 재귀적 분할 기반. |
| `method` | `Literal["suffix", "prefix"]` | `"suffix"` | 컨텍스트 추가 방법. `"suffix"`는 다음 청크의 시작 부분을 현재 청크 끝에 추가, `"prefix"`는 이전 청크의 끝 부분을 현재 청크 시작에 추가. |
| `rules` | `RecursiveRules` | `RecursiveRules()` | 재귀적 오버랩에 사용할 규칙. 기본값은 빈 규칙입니다. |
| `merge` | `bool` | `True` | 컨텍스트를 청크 텍스트에 병합할지 여부. `True`이면 텍스트에 직접 추가, `False`이면 `context` 속성으로만 저장. |
| `inplace` | `bool` | `True` | 청크를 제자리에서 수정할지 여부. `False`이면 청크의 복사본을 생성합니다. |

**리파인 메서드 (`refine`)**

```python
def refine(self, chunks: List[Chunk]) -> List[Chunk]:
    """Refine the chunks based on the overlap."""
    logger.debug(f"Starting overlap refinery for {len(chunks)} chunks with method={self.method}, mode={self.mode}")
    
    # 빈 청크 리스트 체크
    if not chunks:
        return chunks
    
    # 모든 청크가 같은 타입인지 확인
    if len(set(type(chunk) for chunk in chunks)) > 1:
        raise ValueError("All chunks must be of the same type.")
    
    # inplace가 False이면 복사본 생성
    if not self.inplace:
        chunks = [chunk.copy() for chunk in chunks]
    
    # 효과적인 컨텍스트 크기 계산
    effective_context_size = self._get_overlap_context_size(chunks)
    
    # method에 따라 prefix 또는 suffix 오버랩 적용
    if self.method == "prefix":
        refined_chunks = self._refine_prefix(chunks, effective_context_size)
    elif self.method == "suffix":
        refined_chunks = self._refine_suffix(chunks, effective_context_size)
    
    logger.info(f"Overlap refinement complete: added context to {len(refined_chunks)} chunks")
    return refined_chunks
```

**동작 방식:**

1. **Suffix 모드** (`method="suffix"`):
   - 각 청크의 끝에 다음 청크의 시작 부분을 추가합니다.
   - 예: 청크 1의 끝에 청크 2의 시작 부분을 추가하여 문맥을 보강합니다.

2. **Prefix 모드** (`method="prefix"`):
   - 각 청크의 시작에 이전 청크의 끝 부분을 추가합니다.
   - 예: 청크 2의 시작에 청크 1의 끝 부분을 추가하여 문맥을 보강합니다.

3. **Token 모드** (`mode="token"`):
   - 토크나이저를 사용하여 정확한 토큰 수만큼 컨텍스트를 추출합니다.

4. **Recursive 모드** (`mode="recursive"`):
   - 재귀적 규칙을 사용하여 의미 있는 단위로 컨텍스트를 추출합니다.

**성능 최적화:**

- **LRU 캐싱**: 토크나이제이션 작업을 캐싱하여 반복 작업의 성능을 향상시킵니다 (maxsize=8192).
- **캐시 모니터링**: `cache_info()` 메서드로 캐시 통계를 확인할 수 있습니다.
- **캐시 초기화**: `clear_cache()` 메서드로 메모리를 해제할 수 있습니다.

**사용 예시:**

```python
from chonkie.refinery import OverlapRefinery

# 기본 설정 (문자 단위, 25% 오버랩, suffix 모드)
refinery = OverlapRefinery()

# 토큰 기반 오버랩 (50 토큰)
refinery = OverlapRefinery(
    tokenizer="tiktoken",
    context_size=50,
    mode="token",
    method="suffix"
)

# 재귀적 오버랩 (20% 비율)
refinery = OverlapRefinery(
    context_size=0.2,
    mode="recursive",
    method="prefix",
    merge=True,
    inplace=False  # 원본 청크 보존
)

# 청크에 오버랩 적용
refined_chunks = refinery.refine(chunks)
```

**주의사항:**

- `context_size`가 float인 경우, 최대 청크 크기의 비율로 계산됩니다.
- `merge=True`일 때, 컨텍스트가 청크 텍스트에 직접 추가되므로 원본 위치 정보(`start_index`, `end_index`)는 변경되지 않습니다.
- `inplace=False`로 설정하면 원본 청크를 보존하고 복사본을 반환합니다.


In [ ]:
# ============================================================================
# Refinery 활용 - OverlapRefinery, EmbeddingsRefinery
# ============================================================================
from chonkie.refinery import EmbeddingsRefinery, OverlapRefinery

# 기본 청크 사용 (이전에 생성한 chunks_sent)
print(f"\n{'=' * 80}")
print("원본 청크 정보")
print(f"{'=' * 80}")
print(f"원본 청크 수: {len(chunks_sent)}")
print(f"원본 첫 청크 길이: {len(chunks_sent[0].text)} 문자")
print(f"원본 첫 청크 텍스트 (처음 100자): {chunks_sent[0].text[:100]}...")

# ============================================================================
# 1) OverlapRefinery - 다양한 옵션 비교
# ============================================================================
print(f"\n{'=' * 80}")
print("[1/3] OverlapRefinery - 다양한 옵션 비교")
print(f"{'=' * 80}")

# 1-1) 기본 설정 (문자 단위, 25% 오버랩, suffix 모드)
print("\n[1-1] 기본 설정 (문자 단위, context_size=0.25, suffix 모드)")
refinery_basic = OverlapRefinery(
    tokenizer="character",
    context_size=0.25,  # 최대 청크 크기의 25%
    mode="token",
    method="suffix",
    merge=True,
    inplace=True,
)
refined_basic = refinery_basic.refine(
    chunks_sent.copy() if hasattr(chunks_sent, "copy") else chunks_sent[:]
)
print(f"   리파인 후 첫 청크 길이: {len(refined_basic[0].text)} 문자")
print(f"   길이 증가: {len(refined_basic[0].text) - len(chunks_sent[0].text)} 문자")
print(f"   첫 청크 텍스트 (처음 150자): {refined_basic[0].text[:150]}...")

# 1-2) 토큰 기반 오버랩 (고정 토큰 수)
print("\n[1-2] 토큰 기반 오버랩 (context_size=50 토큰, suffix 모드)")
try:
    refinery_token = OverlapRefinery(
        tokenizer="tiktoken",  # 또는 다른 토크나이저
        context_size=50,  # 고정 토큰 수
        mode="token",
        method="suffix",
        merge=True,
        inplace=False,  # 원본 보존
    )
    refined_token = refinery_token.refine(
        chunks_sent.copy() if hasattr(chunks_sent, "copy") else chunks_sent[:]
    )
    print(f"   리파인 후 첫 청크 길이: {len(refined_token[0].text)} 문자")
    print(f"   첫 청크 텍스트 (처음 150자): {refined_token[0].text[:150]}...")
except Exception as e:
    print(f"   토크나이저 사용 불가: {e}")
    print("   문자 단위로 대체 실행...")
    refinery_token = OverlapRefinery(
        tokenizer="character",
        context_size=50,  # 문자 수로 대체
        mode="token",
        method="suffix",
        merge=True,
        inplace=False,
    )
    refined_token = refinery_token.refine(
        chunks_sent.copy() if hasattr(chunks_sent, "copy") else chunks_sent[:]
    )
    print(f"   리파인 후 첫 청크 길이: {len(refined_token[0].text)} 문자")

# 1-3) Prefix 모드 (이전 청크의 끝 부분을 현재 청크 시작에 추가)
print("\n[1-3] Prefix 모드 (이전 청크의 끝 부분을 현재 청크 시작에 추가)")
refinery_prefix = OverlapRefinery(
    tokenizer="character",
    context_size=0.2,  # 20% 비율
    mode="token",
    method="prefix",  # prefix 모드
    merge=True,
    inplace=False,
)
refined_prefix = refinery_prefix.refine(
    chunks_sent.copy() if hasattr(chunks_sent, "copy") else chunks_sent[:]
)
print(
    f"   리파인 후 두 번째 청크 길이: {len(refined_prefix[1].text) if len(refined_prefix) > 1 else 0} 문자"
)
if len(refined_prefix) > 1:
    print(f"   두 번째 청크 텍스트 (처음 150자): {refined_prefix[1].text[:150]}...")

# 1-4) 캐시 정보 확인 (LRU 캐싱 최적화)
print("\n[1-4] LRU 캐시 정보 확인")
cache_info = refinery_basic.cache_info()
print(f"   토큰 캐시 정보: {cache_info.get('tokens_cache', {})}")
print(f"   카운트 캐시 정보: {cache_info.get('count_cache', {})}")

# ============================================================================
# 2) EmbeddingsRefinery - 임베딩 벡터 부착
# ============================================================================
print(f"\n{'=' * 80}")
print("[2/3] EmbeddingsRefinery - 임베딩 벡터 부착")
print(f"{'=' * 80}")

# 기본 설정으로 임베딩 부착
print("\n[2-1] EmbeddingsRefinery 기본 사용")
refinery_emb = EmbeddingsRefinery(embedding_model=chunkie_openai_embeddins)
refined_emb = refinery_emb.refine(refined_basic)

print(f"   리파인 후 청크 수: {len(refined_emb)}")

# 임베딩 확인
if hasattr(refined_emb[0], "embedding"):
    emb_vec = refined_emb[0].embedding
    print(f"   첫 청크 임베딩 차원: {len(emb_vec)}")
    print(f"   임베딩 벡터 (처음 5개): {emb_vec[:5]}")
    print(f"   임베딩 벡터 타입: {type(emb_vec)}")
    print("   임베딩이 성공적으로 부착되었습니다!")
elif hasattr(refined_emb[0], "metadata") and "embedding" in refined_emb[0].metadata:
    emb_vec = refined_emb[0].metadata.get("embedding")
    print(f"   첫 청크 임베딩 차원: {len(emb_vec)}")
    print(f"   임베딩 벡터 (처음 5개): {emb_vec[:5]}")
    print("   임베딩이 메타데이터에 부착되었습니다!")
else:
    print("    임베딩 속성을 찾을 수 없습니다.")
    print(f"   Chunk 속성: {[attr for attr in dir(refined_emb[0]) if not attr.startswith('_')]}")

# ============================================================================
# 3) 통합 파이프라인 - OverlapRefinery + EmbeddingsRefinery
# ============================================================================
print(f"\n{'=' * 80}")
print("[3/3] 통합 파이프라인 - OverlapRefinery → EmbeddingsRefinery")
print(f"{'=' * 80}")

# 원본 청크 복사 (inplace=False로 원본 보존)
original_chunks = chunks_sent.copy() if hasattr(chunks_sent, "copy") else chunks_sent[:]

# Step 1: OverlapRefinery로 문맥 보강
print("\n[Step 1] OverlapRefinery로 문맥 보강")
refinery_overlap_final = OverlapRefinery(
    tokenizer="character",
    context_size=0.1,  # 10% 오버랩
    mode="token",
    method="suffix",
    merge=True,
    inplace=False,
)
chunks_with_overlap = refinery_overlap_final.refine(original_chunks)
print(f"   원본 첫 청크 길이: {len(original_chunks[0].text)} 문자")
print(f"   오버랩 후 첫 청크 길이: {len(chunks_with_overlap[0].text)} 문자")
print(f"   길이 증가: {len(chunks_with_overlap[0].text) - len(original_chunks[0].text)} 문자")

# Step 2: EmbeddingsRefinery로 임베딩 부착
print("\n[Step 2] EmbeddingsRefinery로 임베딩 부착")
refinery_emb_final = EmbeddingsRefinery(embedding_model=chunkie_openai_embeddins)
chunks_final = refinery_emb_final.refine(chunks_with_overlap)
print(f"   최종 청크 수: {len(chunks_final)}")

# 최종 결과 확인
if hasattr(chunks_final[0], "embedding") and chunks_final[0].embedding is not None:
    print(f"   첫 청크 임베딩 차원: {len(chunks_final[0].embedding)}")
    print(f"   첫 청크 텍스트 길이: {len(chunks_final[0].text)} 문자")
    print("\n  통합 파이프라인 완료!")
    print("   - 오버랩으로 문맥이 보강되었습니다.")
    print("   - 임베딩 벡터가 부착되었습니다.")
    print("   - 벡터 데이터베이스에 upsert할 준비가 되었습니다!")


2025-11-09 17:36:23.339 | DEBUG    | chonkie.refinery.overlap:refine:456 - Starting overlap refinery for 4 chunks with method=suffix, mode=token
2025-11-09 17:36:23.340 | INFO     | chonkie.refinery.overlap:refine:481 - Overlap refinement complete: added context to 4 chunks



원본 청크 정보
원본 청크 수: 4
원본 첫 청크 길이: 151 문자
원본 첫 청크 텍스트 (처음 100자): Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.

첫 번째로 TokenChunke...

[1/3] OverlapRefinery - 다양한 옵션 비교

[1-1] 기본 설정 (문자 단위, context_size=0.25, suffix 모드)
   리파인 후 첫 청크 길이: 188 문자
   길이 증가: 0 문자
   첫 청크 텍스트 (처음 150자): Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.

첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.
빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니...

[1-2] 토큰 기반 오버랩 (context_size=50 토큰, suffix 모드)


2025-11-09 17:36:23.855 | DEBUG    | chonkie.refinery.overlap:refine:456 - Starting overlap refinery for 4 chunks with method=suffix, mode=token
2025-11-09 17:36:23.862 | INFO     | chonkie.refinery.overlap:refine:481 - Overlap refinement complete: added context to 4 chunks
2025-11-09 17:36:23.863 | DEBUG    | chonkie.refinery.overlap:refine:456 - Starting overlap refinery for 4 chunks with method=prefix, mode=token
2025-11-09 17:36:23.863 | INFO     | chonkie.refinery.overlap:refine:481 - Overlap refinement complete: added context to 4 chunks
2025-11-09 17:36:23.864 | DEBUG    | chonkie.refinery.embedding:refine:51 - Starting embedding refinery for 4 chunks


   ⚠️  토크나이저 사용 불가: Tokenizer not found in transformers, tokenizers, or tiktoken
   문자 단위로 대체 실행...
   리파인 후 첫 청크 길이: 238 문자

[1-3] Prefix 모드 (이전 청크의 끝 부분을 현재 청크 시작에 추가)
   리파인 후 두 번째 청크 길이: 207 문자
   두 번째 청크 텍스트 (처음 150자): 빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째로 SentenceChunker는 문장 단위로 분할합니다.
자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.

세 번째로 Rec...

[1-4] LRU 캐시 정보 확인
   토큰 캐시 정보: {'hits': 0, 'misses': 3, 'maxsize': 8192, 'currsize': 3}
   카운트 캐시 정보: {'hits': 0, 'misses': 3, 'maxsize': 8192, 'currsize': 3}

[2/3] EmbeddingsRefinery - 임베딩 벡터 부착

[2-1] EmbeddingsRefinery 기본 사용


2025-11-09 17:36:24.274 | INFO     | chonkie.refinery.embedding:refine:56 - Embedding refinement complete: added embeddings to 4 chunks
2025-11-09 17:36:24.275 | DEBUG    | chonkie.refinery.overlap:refine:456 - Starting overlap refinery for 4 chunks with method=suffix, mode=token
2025-11-09 17:36:24.276 | INFO     | chonkie.refinery.overlap:refine:481 - Overlap refinement complete: added context to 4 chunks
2025-11-09 17:36:24.276 | DEBUG    | chonkie.refinery.embedding:refine:51 - Starting embedding refinery for 4 chunks


   리파인 후 청크 수: 4
   첫 청크 임베딩 차원: 1536
   임베딩 벡터 (처음 5개): [ 0.02146324  0.0103791  -0.01436661  0.02064262  0.01710586]
   임베딩 벡터 타입: <class 'numpy.ndarray'>
   임베딩이 성공적으로 부착되었습니다!

[3/3] 통합 파이프라인 - OverlapRefinery → EmbeddingsRefinery

[Step 1] OverlapRefinery로 문맥 보강
   원본 첫 청크 길이: 188 문자
   오버랩 후 첫 청크 길이: 206 문자
   길이 증가: 18 문자

[Step 2] EmbeddingsRefinery로 임베딩 부착


2025-11-09 17:36:24.760 | INFO     | chonkie.refinery.embedding:refine:56 - Embedding refinement complete: added embeddings to 4 chunks


   최종 청크 수: 4
   첫 청크 임베딩 차원: 1536
   첫 청크 텍스트 길이: 206 문자

  통합 파이프라인 완료!
   - 오버랩으로 문맥이 보강되었습니다.
   - 임베딩 벡터가 부착되었습니다.
   - 벡터 데이터베이스에 upsert할 준비가 되었습니다!


## 5. Chunkie 를 통한 Qdrant Handshake (벡터DB 연동)

Chonkie는 다양한 벡터 데이터베이스와의 통합을 지원합니다.   
여기서는 앞에서부터 계속 활용하던 Qdrant를 사용합니다.  

**Qdrant 서버 시작 (Docker):**
```bash
docker run -d -p 6333:6333 -p 6334:6334 qdrant/qdrant:latest
```

**기능:**
- 컬렉션 자동 생성
- 청크 upsert (색인)
- 벡터 검색

**참고 문서:**
- [Qdrant Handshake](https://docs.chonkie.ai/oss/handshakes/qdrant-handshake)


In [ ]:
# ============================================================================
# Qdrant Handshake 데모 - 색인 및 검색
# ============================================================================
import os

# Qdrant 접속 정보
QDRANT_PORT = 6333
QDRANT_URL = os.getenv("QDRANT_URL", f"http://localhost:{QDRANT_PORT}")

print("\n[1/4] Qdrant 서버 확인")
print(f"   Qdrant URL: {QDRANT_URL}")

from chonkie.handshakes import QdrantHandshake

# QdrantHandshake 초기화
print("\n[2/4] QdrantHandshake 초기화")

# QdrantHandshake 초기화 (embedding_model을 통해 벡터 차원 자동 감지)
handshake = QdrantHandshake(
    url=QDRANT_URL,
    collection_name="chonkie_demo",
    embedding_model=chunkie_openai_embeddins,
)
print("   컬렉션: chonkie_demo")
print(f"   벡터 차원: {handshake.dimension}")

# 청크 upsert (색인)
print("\n[3/4] 청크 색인 (upsert)")

# 리파인된 청크 사용 (임베딩 부착된 청크)
# Chonkie Handshake는 임베딩이 부착된 청크를 기대함
print(f"   Indexing {len(refined_emb)} chunks")

try:
    handshake.write(refined_emb)
    print("    Indexing 완료!")
except Exception as e:
    print(f"   Indexing 중 오류: {e}")

# 검색
print("\n[4/4] 벡터 검색")
query = "청킹 전략의 장단점"
print(f"   쿼리: '{query}'")

try:
    hits = handshake.search(query=query, limit=3)

    print(f"   검색 결과 (상위 {len(hits)}개):")
    for i, hit in enumerate(hits):
        text = hit.get("text", "")
        score = hit.get("score", 0.0)
        print(f"\n   [{i + 1}] Score: {score:.4f}")
        print(f"       {text}")
except Exception as e:
    print(f"    검색 중 오류: {e}")

    # 직접 검색 (query_points 사용 - search는 deprecated)
    from qdrant_client import QdrantClient

    client = QdrantClient(url=QDRANT_URL)

    query_emb = chunkie_openai_embeddins.embed(query)
    results = client.query_points(
        collection_name="chonkie_demo",
        query=query_emb,
        limit=3,
        with_payload=True,
    )
    # query_points 결과를 dict 형태로 변환
    results = [{"id": point.id, "score": point.score, **point.payload} for point in results.points]

    print(f"   검색 결과 (상위 {len(results)}개):")
    for i, result in enumerate(results):
        # query_points 결과는 dict 형태
        text = result.get("text", "")[:100]
        score = result.get("score", 0.0)
        print(f"\n   [{i + 1}] Score: {score:.4f}")
        print(f"       {text}")


2025-11-09 17:44:32.457 | DEBUG    | chonkie.handshakes.qdrant:write:191 - Writing 4 chunks to Qdrant collection: chonkie_demo



[1/4] Qdrant 서버 확인
   Qdrant URL: http://localhost:6333

[2/4] QdrantHandshake 초기화
   컬렉션: chonkie_demo
   벡터 차원: 1536

[3/4] 청크 색인 (upsert)
   Indexing 4 chunks


2025-11-09 17:44:33.763 | INFO     | chonkie.handshakes.qdrant:write:199 - Successfully wrote 4 chunks to Qdrant collection: chonkie_demo
2025-11-09 17:44:33.765 | DEBUG    | chonkie.handshakes.qdrant:search:225 - Searching Qdrant collection: chonkie_demo with limit=3


🦛 Chonkie wrote 4 chunks to Qdrant collection: chonkie_demo
    Indexing 완료!

[4/4] 벡터 검색
   쿼리: '청킹 전략의 장단점'


2025-11-09 17:44:34.245 | INFO     | chonkie.handshakes.qdrant:search:241 - Search complete: found 3 matching chunks


   검색 결과 (상위 3개):

   [1] Score: 0.4111
       Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.

첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.
빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째

   [2] Score: 0.2166
       가장 정교하지만 임베딩 모델이 필요합니다.

마지막으로는 SlumberChunker는 LLM 기반 Contextual Chunking 입니다.

   [3] Score: 0.2012
       빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째로 SentenceChunker는 문장 단위로 분할합니다.
자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.

세 번째로 RecursiveChunker는 계층적으로
세 번째로 RecursiveChunker는 계층적으로 분할합니다.


## 6. Porters (입출력)

청크 데이터를 파일로 저장하고 로드합니다.

**종류:**
- **JSONPorter**: JSON 포맷으로 저장/로드
- **DatasetsPorter**: Hugging Face Datasets 포맷

**참고 문서:**
- [JSONPorter](https://docs.chonkie.ai/oss/porters/json-porter)


In [ ]:
import json
import os

from chonkie.porters import JSONPorter
from chonkie.types import Chunk

# JSONPorter 초기화
output_path = "chonkie_chunks.json"
porter = JSONPorter(lines=False)  # JSON 형식으로 저장

print("\n[1/3] 청크 저장 (export)")
print(f"   저장할 청크 수: {len(refined_emb)}")
print(f"   출력 파일: {output_path}")

try:
    porter.export(refined_emb, file=output_path)

    # 파일 크기 확인
    file_size = os.path.getsize(output_path)
    print("    저장 완료!")
    print(f"   파일 크기: {file_size:,} bytes ({file_size / 1024:.2f} KB)")

    # 저장된 내용 미리보기
    print("\n[2/3] 파일 내용 미리보기")
    with open(output_path, encoding="utf-8") as f:
        content = f.read(500)  # 처음 500자
        print(f"   {content}...")

    # 청크 로드
    print("\n[3/3] 청크 로드 (load)")
    with open(output_path, encoding="utf-8") as f:
        chunks_data = json.load(f)
        reloaded_chunks = [Chunk.from_dict(chunk_dict) for chunk_dict in chunks_data]

    print("    로드 완료!")
    print(f"   로드된 청크 수: {len(reloaded_chunks)}")

    if reloaded_chunks:
        first_chunk = reloaded_chunks[0]
        if hasattr(first_chunk, "text"):
            print(f"   첫 청크 텍스트 (처음 80자): {first_chunk.text[:80]}...")
        else:
            print(f"   첫 청크: {str(first_chunk)[:80]}...")

    print("\n JSONPorter로 청크를 파일로 저장하고 재사용할 수 있습니다.")
    print("   - 실험 결과 저장")
    print("   - 청크 버전 관리")
    print("   - 팀 간 청크 공유")

except Exception as e:
    print(f"   ⚠️  오류: {e}")
    import traceback

    print(f"   상세: {traceback.format_exc()}")

2025-11-09 17:48:19.325 | DEBUG    | chonkie.porters.json:_export_json:44 - Exporting 4 chunks to JSON file: chonkie_chunks.json
2025-11-09 17:48:19.331 | INFO     | chonkie.porters.json:_export_json:47 - Successfully exported 4 chunks to JSON: chonkie_chunks.json



[1/3] 청크 저장 (export)
   저장할 청크 수: 4
   출력 파일: chonkie_chunks.json
    저장 완료!
   파일 크기: 214,126 bytes (209.11 KB)

[2/3] 파일 내용 미리보기
   [
    {
        "id": "chnk_bf80c894bec34f98b36f3cf50775206c",
        "text": "Chonkie\ub294 \ub2e4\uc591\ud55c \uccad\ud0b9 \uc804\ub7b5\uc744 \uc81c\uacf5\ud569\ub2c8\ub2e4. \ubb38\uc7a5 \ub2e8\uc704, \ud1a0\ud070 \ub2e8\uc704,\n\uc7ac\uadc0\uc801 \uad6c\uc870 \uc778\uc2dd, \uc758\ubbf8 \uae30\ubc18 \ubd84\ud560 \ub4f1 \uc5ec\ub7ec \ubc29\ubc95\uc744 \ube44\uad50\ud560 \uc218 \uc788\uc2b5\ub2c8\ub2e4.\n\n\uccab \ubc88\uc9f8\ub85c TokenChunker\ub294 \uace0\uc815\ub41c \ud1a0\ud070 \ud06c\uae30...

[3/3] 청크 로드 (load)
    로드 완료!
   로드된 청크 수: 4
   첫 청크 텍스트 (처음 80자): Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다...

 JSONPorter로 청크를 파일로 저장하고 재사용할 수 있습니다.
   - 실험 결과 저장
   - 청크 버전 관리
   - 팀 간 청크 공유


## 7. Chunkie Utils (유틸리티)

청크 시각화 및 헬퍼 함수를 제공합니다.

**기능:**
- **Visualizer**: 청크 시각화 (텍스트, 그래프)
- **Hubbie**: 편의 함수 모음

**참고 문서:**
- [Visualizer](https://docs.chonkie.ai/oss/utils/visualizer)
- [Hubbie](https://docs.chonkie.ai/oss/utils/hubbie)


In [ ]:
# ============================================================================
# Utils 데모 - Visualizer, Hubbie
# ============================================================================
import os
from pathlib import Path

# 출력 디렉토리 생성
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# ============================================================================
# 1) Visualizer - 청크 시각화
# ============================================================================

try:
    from chonkie.utils import Visualizer

    # 1-1) 터미널 시각화 (print 메서드)
    print("\n[1-1] 터미널 시각화 (Rich 라이브러리 사용)")
    print("-" * 80)
    print("   기본 테마 'pastel'로 처음 5개 청크를 색상으로 시각화합니다.")
    print("   각 청크는 서로 다른 색상으로 표시됩니다.\n")

    visualizer = Visualizer(theme="pastel")
    visualizer.print(chunks_sem[:5])

    print("\n   터미널에서 각 청크가 색상으로 구분되어 표시되었습니다!")

    # 1-2) HTML 파일로 저장 (save 메서드)
    print("\n[1-2] HTML 파일로 저장")
    print("-" * 80)
    html_path = output_dir / "chunks_visualization.html"
    print("   처음 10개 청크를 HTML 파일로 저장합니다.")
    print(f"   저장 경로: {html_path.absolute()}\n")

    visualizer.save(str(html_path), chunks_sem[:10], title="Chonkie 청크 시각화")

    print("\n   HTML 파일이 저장되었습니다!")
    print(f"   브라우저에서 열기: file://{html_path.absolute()}")

    # 1-3) 다양한 테마 시연
    print("\n[1-3] 다양한 테마 비교")
    print("-" * 80)
    print("   여러 테마로 동일한 청크를 시각화하여 비교합니다.\n")

    themes = ["pastel", "tiktokenizer", "ocean_breeze"]
    theme_descriptions = {
        "pastel": "파스텔 톤의 부드러운 색상",
        "tiktokenizer": "Tiktokenizer 스타일의 색상",
        "ocean_breeze": "바다를 연상시키는 시원한 색상",
    }

    for theme in themes:
        print(f"   테마: {theme} ({theme_descriptions[theme]})")
        viz = Visualizer(theme=theme)
        # HTML로 저장하여 테마별 비교 가능하도록
        theme_html_path = output_dir / f"chunks_visualization_{theme}.html"
        viz.save(str(theme_html_path), chunks_sem[:5], title=f"Chonkie 청크 시각화 - {theme} 테마")
        print(f"      저장됨: {theme_html_path.name}")

    print("\n   각 테마별 HTML 파일이 저장되었습니다!")
    print("   브라우저에서 열어서 테마별 차이를 비교해보세요.")

except ImportError as e:
    print("   Visualizer를 import할 수 없습니다.")
    print(f"   오류 상세: {e}")
except Exception as e:
    print(f"    오류: {e}")
    import traceback

    print(f"   상세: {traceback.format_exc()}")

# ============================================================================
# 2) Hubbie - Recipe 관리
# ============================================================================
print("\n" + "=" * 80)
print("[2/2] Hubbie - Recipe 관리")
print("=" * 80)

try:
    from chonkie.utils import Hubbie

    print("\n   Hubbie는 HuggingFace Hub에서 Chonkie Recipe를 가져오는 도구입니다.")
    print("   Recipe는 청킹 설정을 재사용 가능한 형태로 저장한 것입니다.\n")

    # 2-1) Hubbie 인스턴스 생성
    print("[2-1] Hubbie 인스턴스 생성")
    print("-" * 80)
    hubbie = Hubbie()
    print("   Hubbie 인스턴스가 생성되었습니다.")

    # 2-2) Recipe Schema 조회
    print("\n[2-2] Recipe Schema 조회")
    print("-" * 80)
    print("   Recipe의 구조를 정의하는 스키마를 조회합니다.\n")

    schema = hubbie.get_recipe_schema()
    print(f"   스키마 버전: {hubbie.SCHEMA_VERSION}")
    print(f"   스키마 키: {list(schema.keys())[:5]}...")  # 처음 5개 키만 표시
    print("\n   Recipe Schema를 성공적으로 조회했습니다!")
    print("   Recipe는 이 스키마에 따라 구조화되어 있습니다.")

    # 2-3) Recipe 조회 (선택적, 네트워크 연결 필요)
    print("\n[2-3] Recipe 조회 (선택적)")

    try:
        recipe = hubbie.get_recipe(name="default", lang="en")
        print("    Recipe를 성공적으로 조회했습니다!")
        print(f"   Recipe 키: {list(recipe.keys())[:5]}...")  # 처음 5개 키만 표시
        print("\n   Recipe는 청킹 설정을 포함하고 있습니다:")
        if "chunker" in recipe:
            print(f"      - Chunker: {recipe.get('chunker', {}).get('type', 'N/A')}")
        if "refinery" in recipe:
            print(f"      - Refinery: {len(recipe.get('refinery', []))}개")
    except Exception as recipe_error:
        print("    Recipe 조회 실패 (네트워크 연결 필요 또는 Recipe가 존재하지 않음)")
        print(f"   오류: {recipe_error}")
        print("\n    Recipe 조회는 선택적 기능입니다.")
        print("   네트워크 연결이 없어도 Schema 조회는 가능합니다.")

    # 2-4) Pipeline Recipe 조회 (선택적)
    print("\n[2-4] Pipeline Recipe 조회 (선택적)")
    print("-" * 80)
    print("   Hub에서 Pipeline Recipe를 조회합니다.")
    print("   ⚠️  네트워크 연결이 필요합니다.\n")
    print("   💡 Pipeline Recipe는 전체 청킹 파이프라인 설정을 포함합니다.")
    print("   참고: 실제로 존재하는 recipe 이름을 사용해야 합니다.\n")

    try:
        # 일반적인 pipeline recipe 이름 시도
        pipeline_recipe = hubbie.get_pipeline_recipe(name="default")
        print("    Pipeline Recipe를 성공적으로 조회했습니다!")
        print(f"   Pipeline 단계 수: {len(pipeline_recipe.get('steps', []))}")
    except Exception as pipeline_error:
        error_str = str(pipeline_error)

        print(f"   ⚠️  Pipeline Recipe 조회 실패")

        # 404 오류인 경우 더 명확한 안내
        if "404" in error_str or "Not Found" in error_str:
            print(f"   원인: 'default'라는 이름의 Pipeline Recipe가 Hub에 존재하지 않습니다.")
            print(f"\n   💡 해결 방법:")
            print(f"   1. HuggingFace Hub에서 사용 가능한 Pipeline Recipe 확인:")
            print(f"      https://huggingface.co/datasets/chonkie-ai/recipes/tree/main/pipelines")
            print(f"   2. 실제로 존재하는 recipe 이름을 사용하세요.")
            print(f"      예: hubbie.get_pipeline_recipe(name='실제_recipe_이름')")
            print(f"   3. 또는 로컬 파일 경로를 직접 지정할 수 있습니다.")
            print(
                f"      예: hubbie.get_pipeline_recipe(name='recipe', path='./local_recipe.json')"
            )
        else:
            print(f"   오류: {pipeline_error}")

        print(f"\n   💡 Pipeline Recipe 조회는 선택적 기능입니다.")
        print(f"   일반 Recipe 조회와 Schema 조회는 정상적으로 작동합니다.")

except ImportError as e:
    print("    Hubbie를 import할 수 없습니다.")
    print("   의존성 설치: pip install 'chonkie[hub]'")
    print(f"   오류 상세: {e}")
except Exception as e:
    print(f"   ⚠️  오류: {e}")
    import traceback

    print(f"   상세: {traceback.format_exc()}")

# ============================================================================
# 3) 청크 통계
# ============================================================================
print("\n" + "=" * 80)
print("💡 청크 통계")
print("=" * 80)

print(f"   총 청크 수: {len(chunks_sem)}")

chunk_lengths = [len(c.text if hasattr(c, "text") else str(c)) for c in chunks_sem]
if chunk_lengths:
    avg_len = sum(chunk_lengths) / len(chunk_lengths)
    min_len = min(chunk_lengths)
    max_len = max(chunk_lengths)

    print(f"   평균 길이: {avg_len:.1f} 문자")
    print(f"   최소 길이: {min_len} 문자")
    print(f"   최대 길이: {max_len} 문자")

    # 토큰 수 통계 (가능한 경우)
    if hasattr(chunks_sem[0], "token_count") and chunks_sem[0].token_count:
        token_counts = [
            c.token_count for c in chunks_sem if hasattr(c, "token_count") and c.token_count
        ]
        if token_counts:
            avg_tokens = sum(token_counts) / len(token_counts)
            min_tokens = min(token_counts)
            max_tokens = max(token_counts)
            print("\n   토큰 통계:")
            print(f"   평균 토큰 수: {avg_tokens:.1f}")
            print(f"   최소 토큰 수: {min_tokens}")
            print(f"   최대 토큰 수: {max_tokens}")

print("\n" + "=" * 80)
print("Utils 데모 완료!")
print("=" * 80)



[1/2] Visualizer - 청크 시각화

[1-1] 터미널 시각화 (Rich 라이브러리 사용)
--------------------------------------------------------------------------------
   기본 테마 'pastel'로 처음 5개 청크를 색상으로 시각화합니다.
   각 청크는 서로 다른 색상으로 표시됩니다.



Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.

첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.
빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째로 SentenceChunker는 문장 단위로 분할합니다.
자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.

세 번째로 RecursiveChunker는 계층적으로 분할합니다.
문단, 문장, 단어 순서로 구조를 보존합니다.

네 번째로는 SemanticChunker는 의미 유사도 기반으로 분할합니다.
가장 정교하지만 임베딩 모델이 필요합니다.

마지막으로는 SlumberChunker는 LLM 기반 Contextual Chunking 입니다.


   터미널에서 각 청크가 색상으로 구분되어 표시되었습니다!

[1-2] HTML 파일로 저장
--------------------------------------------------------------------------------
   처음 10개 청크를 HTML 파일로 저장합니다.
   저장 경로: /Users/jhj/Desktop/sds_class/Day2/notebooks/outputs/chunks_visualization.html

HTML visualization saved to: file:///Users/jhj/Desktop/sds_class/Day2/notebooks/outputs/chunks_visualization.html

   HTML 파일이 저장되었습니다!
   브라우저에서 열기: file:///Users/jhj/Desktop/sds_class/Day2/notebooks/outputs/chunks_visualization.html

[1-3] 다양한 테마 비교
--------------------------------------------------------------------------------
   여러 테마로 동일한 청크를 시각화하여 비교합니다.

   테마: pastel (파스텔 톤의 부드러운 색상)
HTML visualization saved to: file:///Users/jhj/Desktop/sds_class/Day2/notebooks/outputs/chunks_visualization_pastel.html
      저장됨: chunks_visualization_pastel.html
   테마: tiktokenizer (Tiktokenizer 스타일의 색상)
HTML visualization saved to: file:///Users/jhj/Desktop/sds_class/Day2/notebooks/outputs/chunks_visualization_tiktokenizer.html
      저장됨: chun

v1.schema.json: 0.00B [00:00, ?B/s]

   ✅ Hubbie 인스턴스가 생성되었습니다.

[2-2] Recipe Schema 조회
--------------------------------------------------------------------------------
   Recipe의 구조를 정의하는 스키마를 조회합니다.

   스키마 버전: v1
   스키마 키: ['$schema', 'title', 'type', 'required', 'properties']...

   Recipe Schema를 성공적으로 조회했습니다!
   Recipe는 이 스키마에 따라 구조화되어 있습니다.

[2-3] Recipe 조회 (선택적)


default_en.json: 0.00B [00:00, ?B/s]

    Recipe를 성공적으로 조회했습니다!
   Recipe 키: ['name', 'schema', 'description', 'language', 'metadata']...

   Recipe는 청킹 설정을 포함하고 있습니다:

[2-4] Pipeline Recipe 조회 (선택적)
--------------------------------------------------------------------------------
   Hub에서 Pipeline Recipe를 조회합니다.
   ⚠️  네트워크 연결이 필요합니다.

    Pipeline Recipe 조회 실패 (네트워크 연결 필요 또는 Recipe가 존재하지 않음)
   오류: Could not download pipeline recipe 'default'. Ensure name is correct or provide a valid path. Error: 404 Client Error. (Request ID: Root=1-6910568f-0a9dca7950765a622350bbae;5cb54b07-0432-4c1c-b50a-844c1021d726)

Entry Not Found for url: https://huggingface.co/datasets/chonkie-ai/recipes/resolve/main/pipelines/default.json.

   Pipeline Recipe 조회는 선택적 기능입니다.

💡 청크 통계
   총 청크 수: 3
   평균 길이: 137.0 문자
   최소 길이: 55 문자
   최대 길이: 224 문자

   토큰 통계:
   평균 토큰 수: 97.0
   최소 토큰 수: 25
   최대 토큰 수: 176

Utils 데모 완료!


## 8. End-to-End 파이프라인  
### (청킹 → 리파인 → 임베딩 → Indexing → 검색)

Chunkie 를 활용한 Document Chunking - Indexing 과정 전체 흐름을 하나의 파이프라인으로 통합합니다.

**파이프라인 단계:**
1. 텍스트 입력
2. Chunker로 분할
3. OverlapRefinery로 문맥 보강
4. EmbeddingsRefinery로 임베딩 추가
5. Qdrant Handshake로 Indexing
6. 벡터 검색


In [ ]:
import os

from chonkie.chunker import RecursiveChunker
from chonkie.handshakes import QdrantHandshake
from chonkie.refinery import EmbeddingsRefinery, OverlapRefinery


def end_to_end_pipeline(
    text: str,
    query: str,
    embeddings_model,
    qdrant_url: str = "http://127.0.0.1:6333",
):
    """
    E2E RAG 파이프라인

    Args:
        text: 입력 텍스트
        query: 검색 쿼리
        embeddings_model: 임베딩 모델 (OpenAIEmbeddings 등)
        qdrant_url: Qdrant 서버 URL

    Returns:
        검색 결과 리스트
    """
    print(f"\n입력 텍스트 길이: {len(text)} 문자")

    # 1) 청킹
    print("\n[1/5] 청킹 (RecursiveChunker)")
    chunker = RecursiveChunker(chunk_size=256, min_characters_per_chunk=24)
    chunks = chunker.chunk(text)
    print(f"   생성된 청크 수: {len(chunks)}")

    # 2) 오버랩 리파인
    print("\n[2/5] 오버랩 리파인 (OverlapRefinery)")
    overlap_refinery = OverlapRefinery(mode="recursive")
    chunks_refined = overlap_refinery.refine(chunks)
    print(f"   리파인 후 청크 수: {len(chunks_refined)}")

    # 3) 임베딩 보강
    print("\n[3/5] 임베딩 보강 (EmbeddingsRefinery)")
    embeddings_refinery = EmbeddingsRefinery(embedding_model=embeddings_model)
    chunks_with_embeddings = embeddings_refinery.refine(chunks_refined)
    print("   임베딩 완료")

    # 4) Qdrant 색인
    print("\n[4/5] Qdrant 색인 (QdrantHandshake)")

    try:
        handshake = QdrantHandshake(
            url=qdrant_url,
            collection_name="chonkie_e2e_pipeline",
            embedding_model=embeddings_model,
        )

        # Indexing (Handshake API 사용 시도)
        try:
            handshake.write(chunks_with_embeddings)
            print("    Indexing 완료")
        except Exception as e:
            print(f"    Handshake API 오류, Indexing: {e}")

        # 5) 검색
        print("\n[5/5] 벡터 검색")
        print(f"   쿼리: '{query}'")

        try:
            results = handshake.search(query=query, limit=5)
        except Exception:
            print("   검색 결과가 없습니다.")
            return []

        print(f"   검색 결과 (상위 {len(results)}개):")
        for i, hit in enumerate(results):
            text = hit.get("text", "")
            score = hit.get("score", 0.0)

            print(f"\n   [{i + 1}] Score: {score:.4f}")
            print(f"       {text}...")

        return results

    except Exception as e:
        print(f"  Qdrant 연결 실패: {e}")
        print("   Qdrant 서버를 시작하세요:")
        print("   docker run -d -p 6333:6333 -p 6334:6334 qdrant/qdrant:latest")
        return []


# 긴 샘플 텍스트
pipeline_text = sample_text * 5  # 충분한 텍스트 확보

# 쿼리
pipeline_query = "의미 기반 청킹의 장점은?"

# 실행
results = end_to_end_pipeline(
    text=pipeline_text,
    query=pipeline_query,
    embeddings_model=chunkie_openai_embeddins,
    qdrant_url=QDRANT_URL,
)
results

2025-11-09 18:01:27.945 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized RecursiveChunker
2025-11-09 18:01:27.945 | DEBUG    | chonkie.chunker.recursive:chunk:368 - Starting recursive chunking for text of length 2055
2025-11-09 18:01:27.945 | INFO     | chonkie.chunker.recursive:chunk:370 - Created 9 chunks using recursive chunking
2025-11-09 18:01:27.946 | DEBUG    | chonkie.refinery.overlap:refine:456 - Starting overlap refinery for 9 chunks with method=suffix, mode=recursive
2025-11-09 18:01:27.946 | INFO     | chonkie.refinery.overlap:refine:481 - Overlap refinement complete: added context to 9 chunks
2025-11-09 18:01:27.946 | DEBUG    | chonkie.refinery.embedding:refine:51 - Starting embedding refinery for 9 chunks



입력 텍스트 길이: 2055 문자

[1/5] 청킹 (RecursiveChunker)
   생성된 청크 수: 9

[2/5] 오버랩 리파인 (OverlapRefinery)
   리파인 후 청크 수: 9

[3/5] 임베딩 보강 (EmbeddingsRefinery)


2025-11-09 18:01:28.487 | INFO     | chonkie.refinery.embedding:refine:56 - Embedding refinement complete: added embeddings to 9 chunks
2025-11-09 18:01:28.543 | DEBUG    | chonkie.handshakes.qdrant:write:191 - Writing 9 chunks to Qdrant collection: chonkie_e2e_pipeline


   임베딩 완료

[4/5] Qdrant 색인 (QdrantHandshake)


2025-11-09 18:01:33.224 | INFO     | chonkie.handshakes.qdrant:write:199 - Successfully wrote 9 chunks to Qdrant collection: chonkie_e2e_pipeline
2025-11-09 18:01:33.225 | DEBUG    | chonkie.handshakes.qdrant:search:225 - Searching Qdrant collection: chonkie_e2e_pipeline with limit=5


🦛 Chonkie wrote 9 chunks to Qdrant collection: chonkie_e2e_pipeline
    Indexing 완료

[5/5] 벡터 검색
   쿼리: '의미 기반 청킹의 장점은?'


2025-11-09 18:01:33.965 | INFO     | chonkie.handshakes.qdrant:search:241 - Search complete: found 5 matching chunks


   검색 결과 (상위 5개):

   [1] Score: 0.3074
       Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.

첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.
빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째로 SentenceChunker는 문장 단위로 분할합니다.
자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.

세 번째로 RecursiveChunker는 계층적으로 분할합니다.
...

   [2] Score: 0.2487
       
마지막으로는 SlumberChunker는 LLM 기반 Contextual Chunking 입니다.Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.

첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.
빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째로 SentenceChunker는 문장 단위로 분할합니다.
자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.

...

   [3] Score: 0.2487
       
마지막으로는 SlumberChunker는 LLM 기반 Contextual Chunking 입니다.Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,
재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.

첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.
빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.

두 번째로 SentenceChunker는 문장 단위로 분할합니다.
자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.

...

   [4] Score: 0.2337
       
네 번째로는 SemanticChunker는 의미 유사도 

[{'id': '101f2f4b-a129-502a-bc68-5f405af31a37',
  'score': 0.3074306,
  'text': 'Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,\n재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.\n\n첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.\n빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.\n\n두 번째로 SentenceChunker는 문장 단위로 분할합니다.\n자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.\n\n세 번째로 RecursiveChunker는 계층적으로 분할합니다.\n',
  'start_index': 0,
  'end_index': 224,
  'token_count': 262},
 {'id': '99cf4d5e-2a13-5e83-8f73-193d25ed3979',
  'score': 0.24867697,
  'text': '\n마지막으로는 SlumberChunker는 LLM 기반 Contextual Chunking 입니다.Chonkie는 다양한 청킹 전략을 제공합니다. 문장 단위, 토큰 단위,\n재귀적 구조 인식, 의미 기반 분할 등 여러 방법을 비교할 수 있습니다.\n\n첫 번째로 TokenChunker는 고정된 토큰 크기로 분할합니다.\n빠르고 예측 가능하지만 의미 경계를 무시할 수 있습니다.\n\n두 번째로 SentenceChunker는 문장 단위로 분할합니다.\n자연스러운 경계를 유지하지만 크기가 불균등할 수 있습니다.\n\n',
  'start_index': 1589,
  'end_index': 1835,
  'token_count': 280},
 {'id': '0ae45e21-4143-5265-9923-2ac4b8760089',
  'score': 0.24867697,
  'text': '\n마지막으로는 SlumberChunker는 LLM 기반 Contextual Chunking 입니다.C

## 학습 내용 요약

1. **설치 및 환경 설정**
   - Chonkie, OpenAI, Qdrant 클라이언트 설치
   - 환경 변수 설정 (OPENAI_API_KEY 등)

2. **Embeddings**
   - OpenAIEmbeddings: OpenAI 임베딩 모델 사용
   - TEI CustomEmbeddings: 온프레미스 임베딩 서버 연동

3. **Chunkers (청킹 전략)**
   - TokenChunker: 고정 토큰 크기
   - SentenceChunker: 문장 단위
   - RecursiveChunker: 계층적 분할
   - SemanticChunker: 의미 유사도 기반
   - CodeChunker (Experimental): 코드 전용

4. **Refinery (문맥 보강)**
   - OverlapRefinery: 인접 청크 오버랩
   - EmbeddingsRefinery: 임베딩 벡터 부착

5. **Handshakes (벡터DB 연동)**
   - QdrantHandshake: Qdrant 색인 및 검색

6. **Porters (입출력)**
   - JSONPorter: JSON 저장/로드

7. **Utils (유틸리티)**
   - Visualizer: 청크 시각화
   - Hubbie: 편의 함수

8. **E2E 파이프라인**
   - 청킹 → 리파인 → 임베딩 → 색인 → 검색


In [ ]:
# TODO: 직접 E2E 파이프라인을 구현해볼까요?